In [4]:
import numpy as np
import pandas as pd
import requests

In [120]:
from datetime import datetime, timedelta

begging_of_the_year = datetime.now().replace(month=1, day=1).strftime('%Y-%m-%d')

yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
begging_of_the_year, yesterday

('2023-01-01', '2023-02-19')

In [121]:
hltv_players = f'https://www.hltv.org/stats/players?startDate={begging_of_the_year}&endDate={yesterday}'
hltv_players

'https://www.hltv.org/stats/players?startDate=2023-01-01&endDate=2023-02-19'

In [122]:
response = requests.get(hltv_players)

data_link = f'../data/players.html'

with open(data_link, 'w') as f:
    f.write(response.text)

players = pd.concat(pd.read_html(data_link)).rename(columns={'Rating2.0': 'Rating'})
players

,Player,Teams,Maps,Rounds,K-D Diff,K/D,Rating
0,felps,NaN,34,868,144,1.25,1.28
1,XANTARES,NaN,48,1362,221,1.25,1.28
2,Nertz,NaN,32,838,136,1.25,1.27
3,Kvem,NaN,35,923,160,1.26,1.26
4,Norwi,NaN,42,1121,201,1.28,1.26
...,...,...,...,...,...,...,...
234,snapy,NaN,30,823,-151,0.75,0.87
235,MiGHTYMAX,NaN,32,838,-116,0.79,0.86
236,TaZ,NaN,69,1865,-323,0.75,0.85
237,simix,NaN,32,849,-133,0.78,0.85


Well, no teams... Let's try another approach.

In [123]:
from bs4 import BeautifulSoup

with open(data_link) as f:
    soup = BeautifulSoup(f.read())

rows = soup.find('table').find_all(class_='teamCol')

players['Teams'] = [row['data-sort'] for row in rows if row.has_attr('data-sort')]
players

,Player,Teams,Maps,Rounds,K-D Diff,K/D,Rating
0,felps,Fluxo,34,868,144,1.25,1.28
1,XANTARES,Eternal Fire,48,1362,221,1.25,1.28
2,Nertz,Endpoint,32,838,136,1.25,1.27
3,Kvem,IKLA,35,923,160,1.26,1.26
4,Norwi,Aurora,42,1121,201,1.28,1.26
...,...,...,...,...,...,...,...
234,snapy,GTZ,30,823,-151,0.75,0.87
235,MiGHTYMAX,Endpoint,32,838,-116,0.79,0.86
236,TaZ,HONORIS,69,1865,-323,0.75,0.85
237,simix,EC Brugge,32,849,-133,0.78,0.85


Much better! Let's group by Teams now

In [124]:
players.groupby('Teams').sum('Rating').sort_values('Rating', ascending=False)

,Maps,Rounds,K-D Diff,K/D,Rating
Teams,,,,,
MIBR,220,5695,679,6.05,5.85
Fluxo,170,4340,470,5.92,5.71
9INE,295,8010,571,5.59,5.60
Paquetá,160,4355,307,5.58,5.58
Aurora,210,5605,339,5.49,5.56
Young Ninjas,165,4379,252,5.49,5.55
iNation,260,6895,414,5.50,5.52
Monte,258,7088,327,5.38,5.50
IKLA,172,4532,278,5.52,5.48


Interesting. Who plays in Tricked team?

In [125]:
players[players['Teams'] == 'Natus Vincere']

,Player,Teams,Maps,Rounds,K-D Diff,K/D,Rating
